# Chapter 3 - Functions

Functions are the first line of organization in any program. Writing them well is the topic of this chapter.

In [1]:
def foo():
    ...

### Small

The first rule of functions is that they should be small. The second rule is that they should be smaller than that.

## Do one thing

Functions should do just one thing. They should do it well. They should do it only.

A way to know that a function is doing more than “one thing” is if you can extract another function from it with a name that is not merely a restatement of its implementation.

## Sections within Functions

Functions that are divided into many sections, probably us doing more than one thing. Functions that do one thing can’t be divided into sections.

## One level of abstraction per Function

In order to make sure our functions are doing one thing , we need to make sure that the statements are all at the same level of abstraction. Mixing levels of abstraction within a functions is always confusing.

## Reading code from top to bottom.

We want the code to read like a top-down narrative.

## Use Descriptive Names

Don’t be afraid to make a name long. A long descriptive name is better than a short enigmatic name. A long descriptive name is better than a long descriptive comment.

## Function Arguments

0 arg - niladic (ideal)

1 arg - monodic

2 args - dyadic 

3 - triadic

4+ - polyadic

## Flag Arguments

Flag arguments are ugly. Passing a boolean into a function is a truly terrible practice. It immediately complicates the signature of the method, loudly proclaiming that this function does more than one thing. It does one thing if the flag is true and another if the flag is false!

In [10]:
def foo(some_flag: bool):
    if flag:
        # Do something
        ...
    else:
        # Do another thing
        ...

## Argument Objects

When a function seems to need more than two or three arguments, it is likely that some of those arguments ought to be wrapped into a class of their own.

In [4]:
def make_circle(x: float, y: float, radius: float):
    ...

In [5]:
from dataclasses import dataclass


@dataclass
class Point:
    x: float
    y: float
    
    

def make_circle(center: Point, radius: float):
    ...

## Verbs and Keywords

Choosing good names for a function can go a long way toward explaining the intent of the function and the order and intent of the arguments. In the case of a monad, the function and argument should form a very nice verb/noun pair.

In [8]:
# From
# Whatever this “name” thing is, it is being “written.”
def write(name: str):
    ...
    
# To
# Tells us that the “name” thing is a “ﬁeld.”
def write_field(name: str):
    ...
    
    
# Another example
def assert_equals(expected: any,  actual: any) -> bool:
    ...

# This strongly mitigates the problem of having to remember the ordering of the arguments.
def assert_expected_equal_actual(expected: any,  actual: any) -> bool:
    ...

## Have No Side Effects

Side effects are lies. Your function promises to do one thing, but it also does other hidden things. Sometimes it will make unexpected changes to the variables of its own class. Sometimes it will make them to the parameters passed into the function or to system globals. In either case they are devious and damaging

In [ ]:
class User:
    def get_phrase_encoded_by_password(self, password: str):
        ...

class UserGateway:
    def find_user_by_name(user_name: str) -> User:
        ...
    
    
class Cryptographer:
    def decrypt(self, coded_phrase, password):
        ...
        
        
class UserValidator:
    def __init__(self, cryptographer: Cryptographer):
        self.cryptographer = cryptographer
    
    def check_password(self, user_name: string, password: str) -> bool:
        user = UserGateway().find_user_by_name(user_name=user_name)
        
        if user:
            coded_phrase = user.get_phrase_encoded_by_password(password=password)
            phrase = self.cryptographer.decrypt(coded_phrase, password)
            
            if "valid_password" == phrase:
                Session.initialize()
                return True
            
        return False
            

The side effect is the call to Session.initialize() , of course. The checkPassword func-
tion, by its name, says that it checks the password. The name does not imply that it initializes the session.

This side effect creates a temporal coupling. That is, checkPassword can only be
called at certain times (in other words, when it is safe to initialize the session). If it is
called out of order, session data may be inadvertently lost. Temporal couplings are con-
fusing, especially when hidden as a side effect. If you must have a temporal coupling,
you should make it clear in the name of the function. In this case we might rename the
function checkPasswordAndInitializeSession , though that certainly violates “Do one
thing.”

## Command Query Separation

Functions should either do something or answer something, but not both. Either your function should change the state of an object, or it should return some information about that object. Doing both often leads to confusion.

In [3]:
"""
Imagine this from the point of view of the reader. 
What does it mean? 
Is it asking whether the attribute was previously set to the object? 
Or is it asking whether the attribute was successfully set to the object? 
It’s hard to infer the meaning from the call because it’s not clear whether the word “setattr” is a verb or an adjective.
"""

# From
def set_attribute(obj: any, attribute: str, value: any):
    if setattr(obj, attribute, value):
        ...
        
        
# To
def set_attribute_if_exists(obj: any, attribute: str, value: any):
    if hasattr(obj, attribute):
        setattr(obj, attribute, value)

## Prefer Exceptions to Returning Error Codes

Returning error codes from command functions is a subtle violation of command query separation. It promotes commands being used as expressions in the predicates of if statements.

In [11]:
# From
OK_CODE = 'OK'
ERROR_CODE = 'ERROR'

def get_file_content(path: str):
    try:
        with open(path, 'r') as f:
            return f.read()
    except FileNotFoundError as e:
        return ERROR_CODE
    except Exception as e:
        return ERROR_CODE
    
# usage
file =  get_file_content(path='/home/unclebob/desktop/my_file.txt') 

if file <> ERROR_CODE:
    ...
    
    
# To
class OpenFileError(Exception):
    def __init__(self, message: str):
        super().__init__(message)

def get_file_content(path: str):
    try:
        with open(path, 'r') as f:
            return f.read()
    except Exception as e:
        raise OpenFileError(f"Error while opening file. Details: {e}")

## Error Handling Is One Thing

Functions should do one thing. Error handing is one thing. Thus, a function that handles errors should do nothing else. This implies (as in the example above) that if the keyword try exists in a function, it should be the very first word in the function and that there should be nothing after the catch/finally blocks.